In [23]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [24]:
dataset = pd.read_csv('labeled_10k.csv') 
print("Dataset Loaded : Done")

Dataset Loaded : Done


In [25]:
from sklearn.preprocessing import LabelEncoder

df = dataset
dummies = pd.get_dummies(df.country)
merged = pd.concat([df,dummies], axis='columns')

print("Data Processing 1 : Done")

Data Processing 1 : Done


In [26]:
final = merged.drop(['country', 'SWE'],axis='columns')

Y_agency = final.agency
Y_social = final.social


X = final.drop('agency',axis='columns')
X = X.drop('social',axis='columns')

dummies = pd.get_dummies(X.concepts)
X = pd.concat([X,dummies], axis='columns')

X = X.drop(['gender', 'hmid', 'concepts', 'animals', 'married', 'parenthood', 'reflection', 'duration'],axis='columns')

print("Data Processing 2 : Done")

Data Processing 2 : Done


In [27]:
#X["age"] = apply(pd.to_numeric(X["age"]))

X[["age"]] = X[["age"]].apply(pd.to_numeric,  errors='coerce')

print("Data Processing 3 : Done")

Data Processing 3 : Done


In [28]:
tmp = X
tmp = tmp.drop(['moment'],axis='columns')

print("Data Processing 4 : Done")

Data Processing 4 : Done


In [29]:
#df1 = dataset[['moment']]
import nltk
from nltk import ngrams
from collections import defaultdict 

moment_List = dataset['moment'].values
data = []
data_process = []
count_data = defaultdict(int)

for count,value in enumerate(moment_List):
    Four_grams = ngrams(value.split(), 4)
    token_freq = nltk.FreqDist(Four_grams)
    for token in token_freq.keys():
        str1=""
        for val in token:
            str1 += val+" "
        data_process.append(str1)
        data.append((count,str1))
        count_data[count] += 1

print("Prepare Anagram List : Done")

Prepare Anagram List : Done


In [30]:
agency      = dataset['agency'].values
social      = dataset['social'].values

for n, i in enumerate(agency):
    if i == 'no':
        agency[n]=0
    else:
        agency[n]=1
        
for n, i in enumerate(social):
    if i == 'no':
        social[n]=0
    else:
        social[n]=1
        
print("Agency Value set : Done")
print("Social Value set : Done")

Agency Value set : Done
Social Value set : Done


In [31]:
column_values = pd.Series(data_process)
df = pd.DataFrame(data) 

y_agency=[]
for i,j in zip(count_data.values(), agency):
    for val in range(0,i):
        y_agency.append(j)
        
y_social=[]
for i,j in zip(count_data.values(), social):
    for val in range(0,i):
        y_social.append(j)

print("Y_Agency Prepared : Done")
print("Y_Social Prepared : Done")


Y_Agency Prepared : Done
Y_Social Prepared : Done


In [32]:
def split_list(a_list, y):
    return a_list[:y], a_list[y+1:]


x = int(0.2*len(y_agency))+1
y = int(len(y_agency) - 0.2*len(y_agency))
y_agency_train, y_agency_test = split_list(y_agency,y)

x_train_, x_test_ = split_list(data_process, y)

y_social_train, y_social_test = split_list(y_social, y)

print("done")

done


In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf = TfidfVectorizer(min_df=1, stop_words='english')
x_train = tf_idf.fit_transform(x_train_)
x_test = tf_idf.transform(x_test_)

x_train.toarray()

print("Vectorizer : Done")

Vectorizer : Done


In [34]:
# Predicting Agency

from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
# y_agency=y_agency.astype('int')
mnb.fit(x_train.toarray(), y_agency_train)

y_pred = mnb.predict(x_test)

count=0
for i in range(len(y_pred)):
    if y_pred[i] == y_agency_test[i]:
        count += 1

print("Naive Bayes Agency : ",count/len(y_pred) * 100,"% accuracy")

print("Naive Bayes Model Fit : Done")

Naive Bayes Agency :  72.54830808309897 % accuracy
Naive Bayes Model Fit : Done


In [35]:
# Predicting Social

from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
# y_agency=y_agency.astype('int')
mnb.fit(x_train.toarray(), y_social_train)

y_pred = mnb.predict(x_test)

count=0
for i in range(len(y_pred)):
    if y_pred[i] == y_social_test[i]:
        count += 1

print("Naive Bayes : ",count/len(y_pred) * 100,"% accuracy")

print("Naive Bayes Model Fit : Done")

Naive Bayes :  50.19051075024947 % accuracy
Naive Bayes Model Fit : Done
